# 13 - RandomForest + NewDummies

Após um resultado um pouco melhor no `DecisionTree` com o novo dataset com dummies, realizarei uma nova tentativa com o `RandomForest`.

## Preparando o ambiente

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, GroupKFold, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler
from IPython.display import display, clear_output

In [2]:
SEED = 5
np.random.seed(SEED)

## Carregando os dados

In [3]:
treino = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed_v2/train_dummies_2.csv')
teste = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed_v2/test_dummies_2.csv')
Xvalidation = teste.drop(columns=['PassengerId'])
treino.head(2)

,Survived,Pclass,Age,SibSp,Parch,Relateds,possui_cabine,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22,1,0,1,0,0,1,0,0,1
1,1,1,38,1,0,1,1,1,0,1,0,0


## Separando treino e teste

In [4]:
X = treino.drop(columns=['Survived'])
y = treino['Survived']

treino_x, teste_x, treino_y, teste_y = train_test_split(X, y, test_size = 0.2, stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 712 elementos e testaremos com 179 elementos


## Criando a `RandomForest`

In [5]:
def rf_grid_search(treino_x, treino_y, teste_x, teste_y):
    n_estimators = [25, 50, 100, 300, 500]
    criterion = ["gini", "entropy"]
    max_depth = range(3, 100, 4)
    min_samples_split = [4, 8, 16]
    min_samples_leaf = [4, 8, 16]
    resultados = []

    total = len(n_estimators) * len(criterion) * len(max_depth) * len(min_samples_split) * len(min_samples_leaf)
    counter = 0

    for c in criterion:
        for md in max_depth:
            for ms in min_samples_split:
                for ml in min_samples_leaf:
                    for ne in n_estimators:
                        modelo = RandomForestClassifier(n_estimators = ne, criterion = c, max_depth = md, min_samples_split = ms, min_samples_leaf = ml)
                        modelo.fit(treino_x, treino_y)
                        train_score = modelo.score(treino_x, treino_y)
                        test_score = modelo.score(teste_x, teste_y)
                        d = {
                            "criterion": c,
                            "max_depth" : md,
                            "min_samples_split": ms,
                            "min_samples_leaf": ml,
                            "n_estimators": ne,
                            "train_score": train_score,
                            "test_score": test_score
                        }
                        counter += 1
                        resultados.append(d)
                        
                        clear_output(wait=True)
                        display(f'Execução {counter} de {total} - Score: {test_score}')

    return pd.DataFrame(resultados)


In [6]:
resultados = rf_grid_search(treino_x, treino_y, teste_x, teste_y)

'Execução 2250 de 2250 - Score: 0.8100558659217877'

In [10]:
best_result = resultados.sort_values(by=['test_score', 'train_score'], ascending=[False, False]).head(1)
best_result

,criterion,max_depth,min_samples_split,min_samples_leaf,n_estimators,train_score,test_score
360,gini,35,4,4,25,0.849719,0.860335


### Testando no Kaggle com o melhor resultado obtido

In [11]:
best_params = best_result.drop(columns=['test_score', 'train_score']).to_dict(orient='records')[0]
best_params

{'criterion': 'gini',
 'max_depth': 35,
 'min_samples_split': 4,
 'min_samples_leaf': 4,
 'n_estimators': 25}

In [13]:
best_params['random_state'] = SEED
modelo = RandomForestClassifier(**best_params)
modelo.fit(X, y)
predicoes = modelo.predict(Xvalidation)
resultado = pd.DataFrame()
resultado['PassengerId'] = teste['PassengerId']
resultado['Survived'] = predicoes
resultado.to_csv('../../data/submissions/random_forest_new.csv', index=False)

O melhor modelo de `RandomForest` apresentou apenas 77.2% de acurácia. Inferior ao do `DecisionTree` feito anteriormente. Parece que com o dataset dessa forma não vou conseguir obter o resultado maior que 77% na base de validação. O próximo passo será tentar com um SVC - um modelo mais complexo e menos visual. Após isso, caso a acurácia continue semelhante, talvez o ideal seja voltar mais uma vez para o modelo de dados e realizar um novo pré-processamento - talvez baseado em alguma ajuda na internet.

Não vou rodar o modelo com `cross_validation` e com os `Scalers` pois os resultados nas outras tentativas não foram diferentes. Parece uma limitação ou do modelo ou dos dados, visto que ambos os modelos utilizados utilizam a árvore de decisão como base. É necessário, para teste, validar com modelos de proposta diferentes.